In [85]:
import json
import csv
from statistics import mean


import json

# Leitura de Dados
def ler_arquivo(arquivo_json: str) -> dict:
    try:
        with open(arquivo_json, 'r') as arquivo:
            dados = arquivo.read()
            return json.loads(dados)
    except FileNotFoundError:
        print(f"Arquivo '{arquivo_json}' não encontrado.")
        return None
    except json.JSONDecodeError as e:
        print(f"Erro ao decodificar JSON: {str(e)}")
        return []

# Salvar Dados em um Arquivo JSON
def salvar_arquivo(dados: dict, arquivo_json: str) -> None:
    try:
        # Obtém a representação em formato de string dos dados serializados em JSON
        dados_json_string = json.dumps(dados, indent=2)
        # Salva a string JSON no arquivo
        with open(arquivo_json, 'w') as arquivo:
            arquivo.write(dados_json_string)
    except Exception as e:
        raise Exception(f"Erro ao salvar JSON: {str(e)}")




In [86]:

# Manipulação de Dados
# 2. **Manipulação de Dados:**
def mapeamento(dados):
    # Para cada dicionário x na lista dados, retorna um novo dicionário contendo apenas as chaves 'nome' e 'idade'.
    return list(map(lambda x: {'nome': x['nome'], 'idade': x['idade']}, dados))
# Função para filtrar dados por área de atuação
def filtro(dados, area_atuacao):
    return list(filter(lambda x: x['area_atuacao'] == area_atuacao, dados))
# Função para realizar a soma de salários
def reducao(dados):
    return sum([p['salario'] for p in dados])




In [87]:
# 3. **CRUD (Create, Read, Update, Delete):**
# Função para criar  dados 
def criar_pessoa(dados, arquivo_json, **kwargs):
    try:
        # Construir o novo registro usando as chaves fornecidas
        novo_registro = {key: kwargs.get(key) for key in ['nome', 'idade', 'sexo', 'empregado', 'area_atuacao', 'salario']}
        # Adiciona o novo registro ao final
        dados.append(novo_registro)
        # Obtém a representação em formato de string dos dados serializados em JSON
        dados_json_string = json.dumps(dados, indent=2)
        
        # Salva a string JSON atualizada no arquivo JSON
        with open(arquivo_json, 'w') as arquivo:
            arquivo.write(dados_json_string)
            
        print("Registro salvo com sucesso!")
        
    except Exception as e:
        raise Exception(f"Erro ao salvar registro: {str(e)}")

def nova_pessoa(dados, **kwargs):
    try:
        # Construir o novo registro usando as chaves fornecidas
        novo_registro = {key: kwargs.get(key) for key in ['nome', 'idade', 'sexo', 'empregado', 'area_atuacao', 'salario']}
        
        # Criar uma cópia da lista de dados com o novo registro adicionado
        novos_dados = dados.copy()
        novos_dados.append(novo_registro)
        
        # Retornar a nova versão da lista de dados
        return novos_dados

    except Exception as e:
        raise Exception(f"Erro ao criar pessoa: {str(e)}")
        
# Função para ler  dados  de uma pessoa    
def ler_pessoa(dados, nome):
    pessoa = next((p for p in dados if p['nome'] == nome), None)
    if pessoa:
        return pessoa
    else:
        raise Exception(f"Pessoa com nome '{nome}' não encontrada.")

# Função para atualizar os dados
def atualizar_pessoa(dados, nome, **kwargs):
# Busca a pessoa na lista
    for pessoa in dados:
        if pessoa['nome'] == nome:
            # Atualiza os dados da pessoa
            for chave, novo_valor in kwargs.items():
                if chave in pessoa:
                    pessoa[chave] = novo_valor
                else:
                    print(f"A chave {chave} não existe para a pessoa {nome}")

            # Salva os dados atualizados 
            salvar_arquivo(dados, 'dados.json')
             # Mensagem de sucesso
            print(f"Dados atualizados para a pessoa {nome}")
            
            return dados

    # Se a pessoa não foi encontrada, levanta um erro
    raise ValueError(f"Pessoa {nome} não encontrada na lista de dados.")
            
# Função para deletar uma pessoa 
def deletar_pessoa(dados, nome):
    pessoa = ler_pessoa(dados, nome)
    dados.remove(pessoa)
    salvar_arquivo(dados, 'dados.json')
    
def exibir_dados(pessoa: dict) -> str:
    return '\n'.join([f"{chave.capitalize()}: {valor}" for chave, valor in pessoa.items()])     
    


In [88]:

# 5. **Estatísticas Simples:**

# Função para obter a quantidade de pessoas por área
def qtd_pessoas(dados, area_desejada=None):
    # Filtrar para uma área específica se fornecida
    if area_desejada:
        dados = list(filter(lambda x: x['area_atuacao'] == area_desejada, dados))
    # Iterar sobre as áreas únicas presentes nos dados
    areas_unicas = set([pessoa['area_atuacao'] for pessoa in dados if pessoa['area_atuacao']])
    for area in areas_unicas:
        # Filtrar pessoas apenas para a área específica
        pessoas_area = list(filter(lambda x: x['area_atuacao'] == area, dados))
        # Contar a quantidade de pessoas na área
        quantidade = len(pessoas_area)
        
    
    return quantidade

# Média de idade
def calcular_media_idade(dados, area_atuacao): # Função para obter a média de idade conforme área de atuação
    pessoas_area = list(filter(lambda x: x['area_atuacao'] == area_atuacao, dados))
    idades = [pessoa['idade'] for pessoa in pessoas_area]
    media = mean(idades)
    media_arredondada = round(media, 2)
    return media_arredondada

# 6. **Obtenção de Máximo ou Mínimo:**
def max_min(dados, atributo, maximo=True):
    extremos = [(pessoa['nome'], pessoa[atributo]) for pessoa in dados]
    extremo = max(extremos, key=lambda x: x[1]) if maximo else min(extremos, key=lambda x: x[1])
    return extremo

def max_min_por_area(dados, atributo, area_atuacao, maximo=True):
    extremos = [(pessoa['nome'], pessoa[atributo]) for pessoa in dados if pessoa['area_atuacao'] == area_atuacao]
    
    if not extremos:
        return None  # Retorna None se não houver pessoas na área de atuação especificada
    
    extremo = max(extremos, key=lambda x: x[1]) if maximo else min(extremos, key=lambda x: x[1])
    return extremo

# Função para calcular estatísticas 
def estatisticas_por_area(dados, area_atuacao):
    media_idade = calcular_media_idade(dados, area_atuacao)
    max_nome, max_salario = max_min_por_area(dados, "salario", area_atuacao,maximo=True)
    min_nome, min_salario = max_min_por_area(dados, "salario",area_atuacao, maximo=False)
    qtd =  qtd_pessoas(dados,area_atuacao)
    return {
        "area_atuacao": area_atuacao,
        "qtd_pessoas": qtd,
        "media_idade": media_idade,
        "max_salario_nome": max_nome,
        "max_salario": max_salario,
        "min_salario_nome": min_nome,
        "min_salario": min_salario
    }
    
    
    
# Função para exportar estatísticas para um arquivo CSV
def exportar_csv(estatisticas, arquivo_csv):
    try:
        with open(arquivo_csv, mode='w', newline='') as arquivo:
            writer = csv.DictWriter(arquivo, fieldnames=estatisticas.keys())
            writer.writeheader()
            writer.writerow(estatisticas)
    except Exception as e:
        raise Exception(f"Erro ao exportar CSV: {str(e)}")


    
   

In [89]:
def salvar_csv(estatisticas_por_area, arquivo_csv):
    try:
        with open(arquivo_csv, mode='w', newline='') as arquivo:
            writer = csv.writer(arquivo, delimiter=";",lineterminator='\n')
            # Escreve o cabeçalho
            writer.writerow(["Area Atuacao", "Qtd Pessoas", "Media Idade", "Max Nome", "Max Salario", "Min Nome", "Min Salario"])

            # Escreve os dados
            for estatisticas in estatisticas_por_area:
                writer.writerow([estatisticas["area_atuacao"],
                                 estatisticas["qtd_pessoas"],
                                 estatisticas["media_idade"],
                                 estatisticas["max_salario_nome"],
                                 estatisticas["max_salario"],
                                 estatisticas["min_salario_nome"],
                                 estatisticas["min_salario"]])
                print(estatisticas)
            
        print(f"Dados exportados para '{arquivo_csv}' com sucesso.")
    except Exception as e:
        print(f"Erro ao exportar CSV: {str(e)}")

# Função para calcular estatísticas para todas as áreas
def gerar_estatisticas(dados):
    lista_estatisticas = []
    # Obtém todas as áreas únicas de atuação
    areas_de_atuacao = set(dado['area_atuacao'] for dado in dados)
   
    for area_atuacao in areas_de_atuacao:
        estatisticas = estatisticas_por_area(dados, area_atuacao)
        lista_estatisticas.append(estatisticas)

    return lista_estatisticas

In [90]:
def obter_nome(msg=''):
    while True:
        nome_completo = input(msg)
        if nome_completo.replace(" ", "").isalpha():
            return nome_completo
        else:
            print("Por favor, digite um nome completo válido (apenas letras e espaços).")
            
def obter_idade(msg=''):
    while True:
        try:
            idade = int(input(msg))
            if 18 <= idade <= 120:
                return idade
            else:
                print("A idade deve estar entre 18 e 120 anos.")
        except ValueError:
            print("Por favor, digite um número inteiro válido.")    

def obter_sexo(msg=''):
    while True:
        sexo = input(msg).capitalize()
        if sexo in ['Feminino', 'Masculino']:
            return sexo
        else:
            print("Por favor, digite 'Feminino' ou 'Masculino'.")
            
def obter_area_atuacao(msg=''):
    while True:
        entrada = input(msg).strip()
        if len(entrada) >= 2:
            return entrada
        else:
            print("Por favor, digite uma string com pelo menos dois caracteres.")                                
            
def obter_valor(msg='', func=float):
  while True:
        valor = input(msg)
        try:
            valor_convertido = func(valor)
            if valor_convertido > 0:
                return valor_convertido
            else:
                print("O valor deve ser maior que zero.")
        except ValueError:
            print(f'Entrada Inválida! {msg}')
            
            
def obter_inteiro(msg='',func=int):
    while True:
        try:
            return int(input(msg))
        except ValueError:
            print("Por favor, digite um número inteiro válido.")


               
def validar_entrada(nome, idade, sexo, area, salario):
    try:
        # Verifica se o nome foi informado
        if not nome:
            raise ValueError("Nome não informado.")

        # Verifica se a idade é um número inteiro entre 18 e 119
        idade = int(idade)
        if not 18 <= idade < 120:
            raise ValueError("Idade deve estar entre 18 e 119.")

        # Verifica se o sexo é "Feminino" ou "Masculino"
        if sexo not in ["Feminino", "Masculino"]:
            raise ValueError("Sexo deve ser 'Feminino' ou 'Masculino'.")

        # Verifica se o salário é um número real maior que zero
        salario = float(salario)
        if salario <= 0:
            raise ValueError("Salário deve ser maior que zero.")

        # Se todas as verificações passaram, os valores são considerados válidos
        return True, {"nome": nome, "idade": idade, "sexo": sexo, "area": area, "salario": salario}

    except ValueError as e:
        # Se houver um erro, os valores não são válidos
        return False, str(e)
    
    
def obter_entradas():
    try:
        nome = obter_nome("Digite o nome da pessoa: ")
        idade = obter_idade("Digite a idade: ")
        sexo = obter_sexo("Digite o sexo: ")
        area = obter_area_atuacao("Digite a área de atuação: ")
        salario = obter_valor("Digite o salário: ")

        return nome, idade, sexo, area, salario
    except ValueError as e:
        print("Erro ao converter tipo de dado.")
        return None

   

In [91]:
def exibir_menu():
    menu = {
        '1': 'Criar registro',
        '2': 'Atualizar registro',
        '3': 'Deletar registro',
        '4': 'Exibir registro',
        '5': 'Gerar estatísticas',
        '6': 'Sair'
    }
    
    for chave, descricao in menu.items():
        print(f"{chave}. {descricao}")

    print("\n")
    
    opcao = input("Escolha uma opção (1-6): ")
    
    return opcao

def interacao_usuario(dados: dict):
    while True:
        opcao = exibir_menu()

        if opcao == '1':
            dados_pessoa = obter_entradas()

            if dados_pessoa:
                nome, idade, sexo, area, salario = dados_pessoa

            try:
                criar_pessoa(dados, 'dados.json', nome=nome, idade=idade, sexo=sexo, empregado=False, area_atuacao=area, salario=salario)
                print("Registro atualizado com sucesso.")
            except Exception as e:
                print(str(e))

        elif opcao == '2':
            try:
                nome = input("Digite o nome da pessoa: ")
                idade = int(input("Digite a idade: "))
                sexo = input("Digite o sexo: ")
                area = input("Digite a área de atuação: ")
                salario = float(input("Digite o salário: "))
              
                atualizar_pessoa(dados, 'dados.json', nome=nome, idade=idade, sexo=sexo, empregado=False, area_atuacao=area, salario=salario)
                    
                print("Registro atualizado com sucesso.")
                
            except ValueError:
                print("Erro ao converter tipo de dado. Certifique-se de que o salário é um valor numérico.")
            except Exception as e:
                print(str(e))
                
        elif opcao == '3':
            try:
                nome = input("Digite o nome da pessoa para deletar o registro: ")
                deletar_pessoa(dados, nome)
                print("Registro deletado com sucesso.")
                
            except Exception as e:
                print(str(e))
        
        elif opcao == '4':
            try:
                nome = input("Digite o nome da pessoa: ")
                pessoa = ler_pessoa(dados, nome)
                print(exibir_dados(pessoa))
                
            except Exception as e:
                print(str(e))
                          
        elif opcao == '5':
            try:
                arquivo_csv = 'estatisticas.csv'
                estatisticas = gerar_estatisticas(dados)
                salvar_csv(estatisticas, arquivo_csv)
                
                print(f"\nDados estatísticos exportados para '{arquivo_csv}'.")
                
            except Exception as e:
                print(str(e))

        elif opcao == '6':
            print("Encerrando o programa.")
            break

        else:
            print("Opção inválida. Por favor, escolha novamente.")



    
    

In [ ]:
arquivo_json = 'dados.json'
dados = ler_arquivo(arquivo_json)
if dados is not None:
    interacao_usuario(dados)
else:
    # Trate o caso em que o arquivo não foi encontrado ou houve um erro de decodificação JSON
    print("Falha ao carregar dados.")    